# Policy Gradient Methods

In [1]:
import gymnasium as gym
import numpy as np

In [2]:
SEED = 327
rng = np.random.default_rng(SEED)

## REINFORCE

In [3]:
env = gym.make("CartPole-v1")

In [4]:
env.reset(seed=SEED)
env.action_space.seed(SEED)

327

El entorno de CartPole-v1 tiene un espacio de acciones discreto:

| Acción | Descripción                       |
|--------|-----------------------------------|
| 0      | Mover el carro a la izquierda     |
| 1      | Mover el carro a la derecha       |

Por lo tanto, se usará `softmax` como **póliza parametrizable**

La función `softmax` parametrizable es:

$$
\pi(a|s, \theta) = \frac{e^{h(s, a, \theta)}}{\sum_{b} e^{h(s, b, \theta)}}
$$

donde:
$$
h(s, a, \theta)
$$ 

es una **preferencia parametrizable** que puede ser lineal:

$$
h(s, a, \theta) = \theta_a^T s
$$

Además, el espacio de estados es continuo, con los siguientes valores:

| Estado        | Descripción                       | Mínimo | Máximo |
|---------------|-----------------------------------|--------|--------|
| Cart Position | Posición del carro                | -4.8   | 4.8    |
| Cart Velocity | Velocidad del carro               | -Inf   | Inf    |
| Pole Angle    | Ángulo del poste                 | -24°   | 24°    |
| Pole Velocity | Velocidad angular del poste       | -Inf   | Inf    |

In [5]:
S = env.observation_space
A = env.action_space

In [6]:
class SoftmaxPolicy:
    def __init__(self):
        self.S_len = S.shape[0]
        self.A_len = A.n
        
        self.W = np.zeros((self.S_len, self.A_len))
        self.b = np.zeros(self.A_len)

    def logits(self, state):
        return state @ self.W + self.b

    def probs(self, state):
        logits = self.logits(state)
        exp_logits = np.exp(logits - np.max(logits))
        return exp_logits / np.sum(exp_logits)

    def sample(self, state):
        probs = self.probs(state)
        action = rng.choice(self.A_len, p=probs)
        return action, np.log(probs[action])
